# Miscellaneous

Случайные величины, отображения и прочее.

# Преамбула

## Библиотеки

### Math, Numpy, Scipy, Pandas

In [ ]:
import math
import numpy as np
import scipy as sp
import scipy.stats as sps
import scipy.linalg as spl
import pandas as pd

## Распределения

### Базовый класс случайного вектора

In [ ]:
class rv_multidim:
    """Базовый класс для многомерных распределений (структура взята из scipy.stats.rv_continuous)"""

    def __init__(self, momtype=1, xtol=1e-14, badvalue=None, name=None, longname=None, shapes=None, extradoc=None, seed=None):
        if badvalue is None:
            self.badvalue = np.nan
        else:
            self.badvalue = badvalue

        self.name = name
        self.longname = longname
        self.shapes = shapes
        self.extradoc = extradoc
        self.seed = seed

    def rvs(self, *args, **kwds):
        """Сэмплирование."""
        return self._rvs(*args, **kwds)

    def pdf(self, x, *args, **kwds):
        """Функция плотности."""
        return self._pdf(x, *args, **kwds)
    
    def logpdf(self, x, *args, **kwds):
        """Логарифм функции плотности."""
        return np.log(self.pdf(x, *args, **kwds))

    def cdf(self, x, *args, **kwds):
        """Функция распределения."""
        return 1.0 - self.pdf(x, *args, **kwds)

    def logcdf(self, x, *args, **kwds):
        """Логарифм функции распределения."""
        return np.log(self.cdf(x, *args, **kwds))

    def entropy(self, *args, **kwds):
        """Энтропия."""
        return self._entropy(*args, **kwds)

### Несколько полос

In [ ]:
# переделанный класс для строк: поддерживает работу с несколькими массивами

class horizontal_stripes(rv_multidim):
    """Несколько полос с равномерным распределением."""

    def __init__(self, loc_x=np.array([-5, -1, 3]), scale_x=np.array([1, 1, 1]), loc_y=-np.pi, scale_y=2.0*np.pi, seed=None):
        super(some_hor_stripes, self).__init__(name="some distributed stripes", 
                                               longname="some distributed stripes",
                                               shapes="loc_x, scale_x", seed=seed)

        assert len(ranks) != 0, 'Список ranks не должен быть пустым'


        self._loc_x = loc_x
        self._scale_x = scale_x
        self._loc_y = loc_y
        self._scale_y = scale_y


    def _rvs(self, size=1, random_state=None):

        number = sps.randint(low=0, high=3).rvs(1, random_state=None)
        return np.vstack((sps.uniform(loc=self._loc_x[number], scale=self._scale_x[number]).rvs(size, random_state=random_state), 
                         sps.uniform(loc=self._loc_y, scale=self._scale_y).rvs(size, random_state=random_state))).T

    def _entropy(self):
        return np.log(self._loc_x.sum() * self._scale_y)

### Три полосы

In [ ]:
class three_uniform_stripes(rv_multidim):
    """Три полосы с равномерным распределением."""

    def __init__(self, loc_x=-np.pi, scale_x=2.0*np.pi, loc_y1=-5, scale_y1=2, loc_y2=-1, scale_y2=2, loc_y3=3, scale_y3=2, seed=None):
        super(three_uniform_stripes, self).__init__(name="Three uniformly distributed stripes", longname="Three uniformly distributed stripes",
                                                    shapes="loc_x1, scale_x1, loc_x2, scale_x2, loc_x3, scale_x3", seed=seed)
        
        self._loc_x    = loc_x
        self._scale_x  = scale_x
        self._loc_y1   = loc_y1
        self._scale_y1 = scale_y1
        self._loc_y2   = loc_y2
        self._scale_y2 = scale_y2
        self._loc_y3   = loc_y3
        self._scale_y3 = scale_y3

        self._stripe_x  = sps.uniform(self._loc_x,  self._scale_x)
        self._stripe_y1 = sps.uniform(self._loc_y1, self._scale_y1)
        self._stripe_y2 = sps.uniform(self._loc_y2, self._scale_y2)
        self._stripe_y3 = sps.uniform(self._loc_y3, self._scale_y3)

    def _rvs(self, size=1, random_state=None):
        X1 = self._stripe_x.rvs(size, random_state)
        
        X01 = self._stripe_y1.rvs(size, random_state)
        X02 = self._stripe_y2.rvs(size, random_state)
        X03 = self._stripe_y3.rvs(size, random_state)

        X0 = np.concatenate((X01, X02, X03), axis = None)
        X2 = np.random.choice(X0, size = size, replace = False)

        X = np.column_stack((X1,X2))

        return X

    def _entropy(self):
        return np.log((self._scale_y1 + self._scale_y2 + self._scale_y3) * self._scale_x)

### Кольцо Всевластия

In [ ]:
class One_Ring(rv_multidim):
    """Кольцо, полученное из прямоугольника с равномерным распределением."""

    def __init__(self, loc_x=-np.pi, scale_x=2.0*np.pi, less_rad = 1, bigg_rad = 2, seed=None):
        super(One_Ring, self).__init__(name="One Ring that rules them all", longname="One Ring that rules them all",
                                                    shapes="loc_x, scale_x, less_rad, bigg_rad", seed=seed)
        
        assert ((loc_x >= -np.pi) and (scale_x <= 2.0 * np.pi) and (less_rad > 0) and (bigg_rad > less_rad))

        self._loc_x    = loc_x
        self._scale_x  = scale_x
        self._loc_y   = less_rad
        self._scale_y = bigg_rad - less_rad

        self._uniform_x = sps.uniform(self._loc_x, self._scale_x)
        self._uniform_y = sps.uniform(self._loc_y, self._scale_y)

    def _rvs(self, size=1, random_state=None):
        X01 = self._uniform_x.rvs(size, random_state)        
        X02 = self._uniform_y.rvs(size, random_state)
        
        X1 = X02 * np.cos(X01)
        X2 = X02 * np.sin(X01)

        X = np.column_stack((X1,X2))

        return X

    def _entropy(self):
        r = self._loc_y
        delta_r = self._scale_y
        R = r + delta_r
        delta_phi = self._scale_x
        
        return ( R * (np.log(R) - 1.0) - r * (np.log(r) - 1.0) ) / delta_r + np.log(delta_r * delta_phi)

### Случайный вектор с независимо распределёнными координатами

In [ ]:
class rv_id_ensemble(rv_multidim):
    """Случайный вектор с независимо распределёнными координатами."""

    def __init__(self, rv_list):
        self.rv_list = rv_list

    def _rvs(self, size=1, random_state=None):
        XI = []
        
        for i in range(0, len(self.rv_list)):
            XI.append(self.rv_list[i].rvs(size, random_state))
            X = np.column_stack(XI)
        return X

    def _entropy(self):
        entr = 0.0
        
        for i in range(0, len(self.rv_list)):
            entr += self.rv_list[i].entropy()
            
        return entr

## Отображения

### Базовый класс гладкого отображения

In [ ]:
class mapping_smooth:
    """Базовый класс для кусочно гладких отображений"""

    def __init__(self):
        self.input_dim  = None
        self.output_dim = None

    def map(self, x):
        """Отобразить x."""
        return self._map(x)

    def jac(self, x):
        """Матрица Якоби."""
        return self._jac(x)

    def defc(self, x):
        """Коэффициент растяжения."""
        return self._defc(x)

### Ансамбль отображений

In [ ]:
class mapping_ensemble(mapping_smooth):
    """
    Ансамбль отображений.
    """

    def __init__(self, mappings):
        self.input_dim = 0
        self.output_dim = 0
        for i in range(0, len(mappings)):
            self.input_dim += mappings[i].input_dim
            self.output_dim += mappings[i].output_dim
        self.mappings = mappings

    def _map(self, x):
        y = np.zeros(self.output_dim)

        taken_x = 0
        taken_y = 0
        
        for i in range(0, len(self.mappings)):
            y[taken_y:taken_y + self.mappings[i].output_dim] = self.mappings[i].map(x[taken_x:taken_x + self.mappings[i].input_dim])
            
            taken_x += self.mappings[i].input_dim
            taken_y += self.mappings[i].output_dim
            
        return y

    def _jac(self, x):
        jacs = []
        
        taken_x = 0
        
        for i in range(0, len(self.mappings)):    
            jacs.append(self.mappings[i].jac(np.array(x[taken_x:taken_x + self.mappings[i].input_dim])))
            taken_x += self.mappings[i].input_dim        
                
        return spl.block_diag(*jacs)

    def _defc(self, x):
        defcs = []
        
        taken_x = 0
        
        for i in range(0, len(self.mappings)):            
            defcs.append(self.mappings[i].defc(x[taken_x:taken_x + self.mappings[i].input_dim]))
            taken_x += self.mappings[i].input_dim        
              
        defc = 1.0
        
        for i in range(0, len(defcs)):
            defc *= defcs[i]
            
        return defc

### Параллельная система преобразований.

In [ ]:
class mapping_parallel(mapping_smooth):
    """
    Функция, задающая отображение маломерного многообразия на многомерное.
    """

    def __init__(self, mappings):
        self.input_dim = 0
        self.output_dim = 0
        for i in range(0, len(mappings)):
            if (self.input_dim <= mappings[i].input_dim):
                self.input_dim = mappings[i].input_dim
            self.output_dim += mappings[i].output_dim
        self.mappings = mappings

    def _map(self, x):
        y = np.zeros(self.output_dim)

        taken_y = 0
        
        for i in range(0, len(self.mappings)):
            y[taken_y:taken_y + self.mappings[i].output_dim] = self.mappings[i].map(x[0:self.mappings[i].input_dim])
            
            taken_y += self.mappings[i].output_dim
            
        return y

    def _jac(self, x):
        jac = self.mappings[0].jac(np.array(x[0:self.mappings[0].input_dim]))
                
        for i in range(1, len(self.mappings)):    
            jac = np.concatenate((jac, self.mappings[i].jac(np.array(x[0:self.mappings[i].input_dim]))), axis=1)
            
        return jac

    def _defc(self, x):
        J = self.jac(x)
        return np.sqrt(np.linalg.det(J @ J.T))

### Последовательная система преобразований

In [ ]:
class mapping_sequential(mapping_smooth):
    """
    Функция, задающая композицию функций.
    """

    def __init__(self, mappings):
        for i in range(0, len(mappings) - 1):
            assert mappings[i].output_dim == mappings[i+1].input_dim
        
        self.input_dim  = mappings[0].input_dim
        self.output_dim = mappings[-1].output_dim
        self.mappings = mappings

    def _map(self, x):
        y = x
        
        for i in range(0, len(self.mappings)):
            y = self.mappings[i].map(y)
            
        return y

    def _jac(self, x):
        jac = np.identity(self.mappings[0].input_dim)
        y = x
        
        for i in range(0, len(self.mappings)):        
            jac = jac @ self.mappings[i].jac(y)
            y = self.mappings[i].map(y)
            
        return jac

    def _defc(self, x):        
        J = self.jac(x)
        return np.sqrt(np.linalg.det(J @ J.T))

### Преобразование "по отрезкам"

In [ ]:
class mapping_segmented(mapping_smooth):
    """
    Функция, задающая преобразование "по отрезкам".
    """
    
    def __init__(self, mappings, segments):
        assert len(mappings) == len(segments)
        for i in range(len(segments) - 1):
            assert segments[i][1] == segments[i+1][0]
        for mapping in mappings:
            assert mapping.input_dim == 1
            
        self.input_dim  = 1
        self.output_dim = sum([mapping.output_dim for mapping in mappings])
        self.mappings = mappings
        self.segments = segments
        
    def _map(self, x):
        x = x[0]
        taken_y = 0
        y = np.zeros(self.output_dim)
        
        for i in range(len(self.segments)):
            if x < self.segments[i][0]:
                y[taken_y:taken_y + self.mappings[i].output_dim] = self.mappings[i].map(0.0)
            else:
                if x >= self.segments[i][1]:
                    y[taken_y:taken_y + self.mappings[i].output_dim] = self.mappings[i].map(self.segments[i][1] -
                                                                                            self.segments[i][0])
                else:
                    y[taken_y:taken_y + self.mappings[i].output_dim] = self.mappings[i].map(x -
                                                                                            self.segments[i][0])
            
            taken_y += self.mappings[i].output_dim
        
        return y
    
    def _jac(self, x):
        x = x[0]
        taken_y = 0
        jac = np.zeros(self.output_dim)
        
        for i in range(len(self.segments)):
            if self.segments[i][0] <= x < self.segments[i][1]:
                jac[taken_y:taken_y + self.mappings[i].output_dim] = self.mappings[i].jac(x - self.segments[i][0])
            
            taken_y += self.mappings[i].output_dim
            
        return jac
    
    def _defc(self, x):
        x = x[0]
        taken_y = 0
        
        for i in range(len(self.segments)):
            if self.segments[i][0] <= x < self.segments[i][1]:
                return self.mappings[i].defc(x - self.segments[i][0])
            
        return 0.0

### Линейное отображение

In [ ]:
class mapping_linear(mapping_smooth):
    """
    Функция, задающая линейное отображение.
    """

    def __init__(self, matrix):
        (self.output_dim, self.input_dim) = np.shape(matrix)
        self.matrix = np.array(matrix)

    def _map(self, x):
        return self.matrix @ np.array(x)

    def _jac(self, x):
        return self.matrix.T

    def _defc(self, x):
        return np.sqrt(npl.det(self.matrix.T @ self.matrix))

### Круг

In [ ]:
class mapping_circle(mapping_smooth):
    """
    Функция, задающая отображение на единичный круг.
    """

    def __init__(self):
        self.input_dim  = 1
        self.output_dim = 2

    def _map(self, x):
        return np.array([np.cos(x), np.sin(x)])

    def _jac(self, x):
        return np.array([-np.sin(x), np.cos(x)])

    def _defc(self, x):
        return 1.0

### Полуцилиндр

$$
(x',y',z') = g_1(x,y): \quad \begin{matrix} x' = \begin{cases}
\sin(x), \; x \in \left[-\frac{\pi}{2}, \frac{\pi}{2}\right] \\
sign(x), \; else
\end{cases} \\
y' = y \\
z' = \begin{cases}
\cos(x), \; x \in \left[-\frac{\pi}{2}, \frac{\pi}{2}\right] \\
-\left| x - sign(x) \frac{\pi}{2} \right|, \; else
\end{cases}
\end{matrix}
$$

In [ ]:
class mapping_semicylinder(mapping_smooth):
    """
    Функция, задающая отображение плоскости на бесконечный цилиндр с основанием "полукруг + два параллельных луча".
    """

    def __init__(self):
        self.input_dim  = 2
        self.output_dim = 3

    def _map(self, x):
        y = np.zeros(3)

        if ((x[0] >= - np.pi / 2) and (x[0] <= np.pi / 2)):
            y[0] = np.sin(x[0])
            y[1] = x[1]
            y[2] = np.cos(x[0])
        else:
            y[0] = np.sign(x[0])
            y[1] = x[1]
            y[2] = -np.abs(x[0] - (np.sign(x[0]) * np.pi / 2))

        return y

    def _jac(self, x):
        if ((x[0] >= -np.pi / 2) and (x[0] <= np.pi/2)):
            return np.array([[np.cos(x[0]), 0.0, -np.sin(x[0])],
                             [0.0, 1.0, 0.0]])
        else:
            return np.array([[np.cos(x[0]), 0.0, -np.sin(x[0])],
                             [0.0, 1.0, 0.0]])
        
    def _defc(self, x):
        return 1.0

### Бинокль

$$
(x',y',z') = g_2(x,y): \quad \begin{matrix} x' = sign(x)\left(1-\cos(x)\right)\\
y' = y \\
z' = -\sin(x)
\end{matrix}
$$

In [ ]:
class mapping_bicylinder(mapping_smooth):
    """
    Функция, задающая отображение полосы (-2pi, 2pi) x \RR на "двойную трубку".
    """

    def __init__(self):
        self.input_dim  = 2
        self.output_dim = 3

    def _map(self, x):
        assert ((x[0] >= -2.0 * np.pi) and (x[0] <= 2.0 * np.pi))

        y = np.zeros(3)
        
        y[0] = np.sign(x[0]) * (1 - np.cos(x[0]))
        y[1] = x[1]
        y[2] = - np.sin(x[0])

        return y

    def _jac(self, x):
        assert ((x[0] >= -2.0 * np.pi) and (x[0] <= 2.0 * np.pi))

        return np.array([[np.sin(x[0]) * np.sign(x[0]), 0.0, -np.cos(x[0])],
                         [0.0, 1.0, 0.0]])
        
    def _defc(self, x):
        assert ((x[0] >= -2.0 * np.pi) and (x[0] <= 2.0 * np.pi))

        return 1.0

### Сфера ("обратная" проекция Меркатора)

$$
(x',y',z') = g_3(x,y): \quad \begin{matrix} 
x' = \cos x \cos\left(\arctan\left((\sinh y\right)\right) \\
y' = \sin x \cos\left(\arctan\left((\sinh y\right)\right) \\
z' = \sin\left(\arctan\left((\sinh y\right)\right)
\end{matrix}
$$

In [ ]:
class mapping_inverse_Mercator(mapping_smooth):
    """
    Функция, задающая отображение полосы (-pi, pi] x \RR на сферу единичного радиуса.
    """

    def __init__(self):
        self.input_dim  = 2
        self.output_dim = 3

    def _map(self, x):
        assert ((x[0] >= -np.pi) and (x[0] <= np.pi))
        
        y = np.zeros(3)
        
        y[0] = np.cos(x[0]) * np.cos(np.arctan(np.sinh(x[1])))
        y[1] = np.sin(x[0]) * np.cos(np.arctan(np.sinh(x[1])))
        y[2] = np.sin(np.arctan(np.sinh(x[1])))

        return y

    def _jac(self, x):
        assert ((x[0] >= -np.pi) and (x[0] <= np.pi))

        return np.array([[-np.sin(x[0]) * np.cos(np.arctan(np.sinh(x[1]))), np.cos(x[0]) * np.cos(np.arctan(np.sinh(x[1]))), 0.0],
                         [-np.sinh(x[1]) * np.cosh(x[1]) * np.cos(x[0]) / np.power((np.power(np.sinh(x[1]), 2) + 1),(3 / 2)), -np.sinh(x[1]) * np.cosh(x[1]) * np.sin(x[0]) / np.power((np.power(np.sinh(x[1]), 2) + 1),(3 / 2)), -np.cosh(x[1]) / np.power((np.power(np.sinh(x[1]), 2) + 1),(3 / 2))]])
        
    def _defc(self, x):
        assert ((x[0] >= -np.pi) and (x[0] <= np.pi))
        
        return np.cos(np.arctan(np.sinh(x[1]))) / np.cosh(x[1])

### "Плохой" (п.в. дифференцируемый) полуцилиндр

$$
(x',y',z') = g_4(x,y): \quad \begin{matrix} x' = \begin{cases}
\sin(x), \; x \in \left[-\frac{\pi}{2}, \frac{\pi}{2}\right] \\
sign(x) \left(\left|x\right|-\frac{\pi}{2}+1\right), \; else
\end{cases} \\
y' = y \\
z' = \begin{cases}
\cos(x), \; x \in \left[-\frac{\pi}{2}, \frac{\pi}{2}\right] \\
0, \; else
\end{cases}
\end{matrix}
$$

In [ ]:
class mapping_bad_semicylinder(mapping_smooth):
    """
    Функция, задающая отображение плоскости на бесконечный цилиндр с основанием "полукруг + два луча в стороны".
    """

    def __init__(self):
        self.input_dim  = 2
        self.output_dim = 3

    def _map(self, x):
        y = np.zeros(3)

        if ((x[0] >= - np.pi / 2) and (x[0] <= np.pi / 2)):
            y[0] = np.sin(x[0])
            y[1] = x[1]
            y[2] = np.cos(x[0])
        else:
            y[0] = np.sign(x[0]) * (np.abs(x[0]) - (np.pi / 2) + 1)
            y[1] = x[1]
            y[2] = 0

        return y

    def _jac(self, x):
        if ((x[0] >= -np.pi / 2) and (x[0] <= np.pi/2)):
            return np.array([[np.cos(x[0]), 0.0, -np.sin(x[0])],
                             [0.0, 1.0, 0.0]])
        else:
            return np.array([[1.0, 0.0, 0.0],
                             [0.0, 1.0, 0.0]])

    def _defc(self, x):
        return 1.0

### Циклоидоподобные кривые

$$
\left( \begin{matrix}
x \\
y 
\end{matrix} \right) = \left( \begin{matrix}
r \cos \varphi \\
r \sin \varphi
\end{matrix} \right) = \left( \begin{matrix}
2 a \left(1 - \cos \varphi \right) \cos \varphi \\
2 a \left(1 - \cos \varphi \right) \sin \varphi
\end{matrix} \right)
$$
$$
s = 16 a \sin^2\left(\frac{\varphi}{4}\right)
$$

In [ ]:
class mapping_cardioid(mapping_smooth):
    """
    Параметризация кардиоиды длиной (задан радиус окружностей).
    """
    def __init__(self, radius):
        self.input_dim  = 1
        self.output_dim = 2
        self.radius = radius
        
    def _map(self, x):
        y = np.zeros(2)
        
        t = 4 * np.arcsin(np.sqrt(x / (16 * self.radius)))
        
        y[0] = 2 * self.radius * (1 - np.cos(t)) * np.cos(t)
        y[1] = 2 * self.radius * (1 - np.cos(t)) * np.sin(t)
        
        return y
    
    def _jac(self, x):
        jac = np.zeros(2)
        
        jac[0] = (np.sin(4 * np.arcsin(np.sqrt(x / (16 * self.radius)))) * np.cos(4 * np.arcsin(np.sqrt(x / (16 * self.radius)))) - np.sin(4 * np.arcsin(np.sqrt(x / (16 * self.radius)))) * (1 - np.cos(4 * np.arcsin(np.sqrt(x / (16 * self.radius)))))) / np.sqrt((x / self.radius) * (1 - x / (16 * self.radius)))
        jac[1] = (np.power(np.sin(4 * np.arcsin(np.sqrt(x / (16 * self.radius)))),2) + np.cos(4 * np.arcsin(np.sqrt(x / (16 * self.radius)))) * (1 - np.cos(4 * np.arcsin(np.sqrt(x / (16 * self.radius)))))) / np.sqrt((x / self.radius) * (1 - x / (16 * self.radius)))
        
        return jac

    def _defc(self, x):
        return 1.0

$$
\left( \begin{matrix}
x \\
y 
\end{matrix} \right) = \left( \begin{matrix}
r t - r \sin t\\
r - r \cos t
\end{matrix} \right)
$$
$$
s = 4 r \left(1 - \cos\frac{t}{2}\right), \ t \in \left[0, \pi\right]
$$

In [ ]:
class mapping_cycloid(mapping_smooth):
    """
    Параметризация части циклоиды длиной (задан радиус катящейся окружности).
    """
    def __init__(self, radius=1.0):
        self.input_dim  = 1
        self.output_dim = 2
        self.radius = radius
        
    def _map(self, x):
        y = np.zeros(2)
        
        s = x - np.floor(x / (2 * 4 * self.radius)) * 2 * 4 * self.radius
        t = 2 * np.arccos(1 - (s / (4 * self.radius))) + np.floor(x / (2 * 4 * self.radius)) * 2 * np.pi * self.radius
        
        y[0] = self.radius * t - self.radius * np.sin(t)
        y[1] = self.radius - self.radius * np.cos(t)
        
        return y
    
    def _jac(self, x):
        jac = np.zeros(2)
        
        jac[0] = (1 - np.cos(2 * np.arccos(1 - x / (4 * self.radius)))) / (2 * np.sqrt(1 - np.power((1 - x / (4 * self.radius)),2)))
        jac[1] = (2 * np.sin(2 * np.arccos(1 - x / (4 * self.radius)))) / np.sqrt((x * (8 * self.radius - x)) / np.power(self.radius,2))
        
        return jac

    def _defc(self, x):
        return 1.0

$$
x=(r + R) \cos \theta - r \cos \left[(r+R)\frac{\theta}{r}\right]
\\
y=(r + R) \sin \theta - r \sin \left[(r+R)\frac{\theta}{r}\right]
$$
$$
s = 8 R m (1+m) \sin^2 \frac{\theta}{4}
$$
https://encyclopediaofmath.org/wiki/Epicycloid

In [ ]:
class mapping_epicycloid(mapping_smooth):
    """
    Параметризация эпициклоиды длиной (задан радиус фиксированной окружности, число точек возврата).
    """
    def __init__(self, radius=1.0, points=3):
        self.input_dim  = 1
        self.output_dim = 2
        self.radius = radius
        self.points = points
        
    def _map(self, x):
        y = np.zeros(2)
        
        R = self.radius
        m = self.points
        L = 8 * R * (1 + m) / m
        s = x - np.floor(x * m / L) * L / m
        theta = 4 / m * np.arcsin(np.sqrt(s / (8 * m * R * (1 + m)))) + 2 * np.pi / m * np.floor(x * m / L)
        r = R / m
        
        assert ((x >= 0) and (x <= L))
        
        y[0] = (r + R) * np.cos(theta) - r * np.cos((r + R) * theta / r)
        y[1] = (r + R) * np.sin(theta) - r * np.sin((r + R) * theta / r)
        
        return y
    
    def _jac(self, x):
        jac = np.zeros(2)
        
        R = self.radius
        m = self.points
        theta = 4 * np.arcsin(np.sqrt(x / (8 * R * m * (1 + m))))
        r = R / m

        assert ((x >= 0) and (x <= 8 * R * m * (1 + m)))

        jac[0] = (r + R) * (np.sin(theta * (R + r) / r) - np.sin(theta)) / (np.sqrt(2) * m * (m + 1) * R * np.sqrt(x / (m * (m + 1) * R)) * np.sqrt(1 - x / (8 * m * (m + 1) * R)))
        jac[1] = (r + R) * (np.cos(theta) - np.cos(theta * (R + r) / r)) / (np.sqrt(2) * m * (m + 1) * R * np.sqrt(x / (m * (m + 1) * R)) * np.sqrt(1 - x / (8 * m * (m + 1) * R)))
        
        return jac

    def _defc(self, x):
        return 1.0

$$
x=(R - r) \cos \theta + r \cos \left[(R - r)\frac{\theta}{r}\right]
\\
y=(R - r) \sin \theta - r \sin \left[(R - r)\frac{\theta}{r}\right]
$$
$$
s = \frac{8 R (m - 1)}{m^2} \sin^2\frac{\theta}{4}
$$
https://encyclopediaofmath.org/wiki/Hypocycloid

In [ ]:
class mapping_hypocycloid(mapping_smooth):
    """
    Параметризация гипоциклоиды длиной (задан радиус фиксированной окружности, число точек возврата).
    """
    def __init__(self, radius=1.0, points=3):
        self.input_dim  = 1
        self.output_dim = 2
        self.radius = radius
        self.points = points
        
    def _map(self, x):
        y = np.zeros(2)
        
        R = self.radius
        m = self.points
        L = 8 * R * (m - 1) / m
        s = x - np.floor(x * m / L) * L / m
        theta = 4 / m * np.arcsin(np.sqrt(s * (m ** 2) / (8 * R * (m - 1)))) + 2 * np.pi / m * np.floor(x * m / L)
        r = R / m
        
        assert ((x >= 0) and (x <= L))

        y[0] = (R - r) * np.cos(theta) + r * np.cos((R - r) * theta / r)
        y[1] = (R - r) * np.sin(theta) - r * np.sin((R - r) * theta / r)
        
        return y
    
    def _jac(self, x):
        jac = np.zeros(2)
        
        R = self.radius
        m = self.points
        L = 8 * R * (m - 1) / m
        theta = 4 / m * np.arcsin(np.sqrt(x * np.power(m,2) / (8 * R * (m - 1))))
        r = R / m
        
        assert ((x >= 0) and (x <= L))
            
        jac[0] = (r - R) * (np.sin(theta * (R - r) / r) + np.sin(theta)) * (4 * np.sqrt(2) * R * (m - 1) / (np.power(m,2) * np.sqrt(x * R * (m - 1) / np.power(m,2)) * np.sqrt(1 - 8 * x * R * (m - 1) / np.power(m,2))))
        jac[1] = (R - r) * (np.cos(theta) - np.cos(theta * (R - r) / r)) * (4 * np.sqrt(2) * R * (m - 1) / (np.power(m,2) * np.sqrt(x * R * (m - 1) / np.power(m,2)) * np.sqrt(1 - 8 * x * R * (m - 1) / np.power(m,2))))
        
        return jac

    def _defc(self, x):
        return 1.0